# CLUSTERING AND SEGMENTING NEIGHBORHOODS IN CANADA

In [1]:
import pandas as pd
import numpy as np

##  Preprocessing the data

In [2]:
canada_df= pd.read_csv('postal_code_canada.csv')
canada_df = canada_df.dropna(subset=['Borough'])
canada_df= canada_df.drop([290, 291, 292, 293], axis=0)
canada_df= canada_df.mask(canada_df.eq(-1)).ffill(axis=1)
canada_df = canada_df.drop_duplicates(subset=['Neighborhood'])
canada_df.shape

(133, 3)

In [3]:
can_latlong_df= pd.read_csv('Geospatial_Coordinates.csv')
canada_df.rename({'PostalCode':'Postal Code'}, axis='columns', inplace= True)
hood_df = pd.merge(canada_df, can_latlong_df, how='left', on=['Postal Code'])
hood_df.shape

(133, 5)

In [6]:

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /home/karaktah/anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-2.4.1               |           py37_0         543 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         642 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/linux-64::altair-2.4.1-py37_0
  branca             conda-forge/noarch::branca-0.3.1-py_0
  folium             conda-fo

In [11]:
import json # library to handle JSON files

Using Geopy library to get the latitude and longitude of **Toronto**

In [8]:
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode('Toronto, ON')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.653963, -79.387207.


Visualizing the neighborhoods on the map of Toronto

In [9]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(hood_df['Latitude'], hood_df['Longitude'], hood_df['Borough'], hood_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define FourSquare Credentials and version

In [10]:
CLIENT_ID = 'AOJJ0EB1OUDLATTJHDA3GSURAPZOMOFG3QWKL4L034BFCJZK' # your Foursquare ID
CLIENT_SECRET = 'EP5YEA4JJVM2RP40LYWNNBK4DK5ESVTP5JHI23VIQYKV5TXH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AOJJ0EB1OUDLATTJHDA3GSURAPZOMOFG3QWKL4L034BFCJZK
CLIENT_SECRET:EP5YEA4JJVM2RP40LYWNNBK4DK5ESVTP5JHI23VIQYKV5TXH


## Explore Neighborhoods in Manhattan

#### Let's create a function to get nearby venues of all the neighborhoods in Toronto

In [12]:
LIMIT = 20
radius= 500

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Writing the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [14]:

toronto_venues = getNearbyVenues(names=hood_df['Neighborhood'],
                                   latitudes=hood_df['Latitude'],
                                   longitudes=hood_df['Longitude']
                                  )


Parkwoods
Victoria Village
Harbourfront (Toronto)
Regent Park
Lawrence Heights
Lawrence Manor
Queen's Park (Toronto)
Islington Avenue
Rouge, Toronto
Malvern, Toronto
North York
Woodbine Gardens
Parkview Hill
Downtown Toronto
Etobicoke
Islington, Toronto
Princess Gardens
West Deane Park
Highland Creek (Toronto)
Rouge Hill
Port Union, Toronto
Flemingdon Park
Woodbine Heights
St. James Town
Humewood-Cedarvale
Markland Wood
Scarborough, Toronto
Morningside, Toronto
West Hill, Toronto
The Beaches
Berczy Park
Woburn, Toronto
Leaside
Hillcrest Village
Bathurst Manor
Wilson Heights, Toronto
Thorncliffe Park
Dovercourt Village
West Toronto
Scarborough Village
Henry Farm
Northwood Park
York University
East Toronto
Toronto Islands
Union Station (Toronto)
Little Portugal, Toronto
Trinity–Bellwoods
Ionview
Kennedy Park, Toronto
Bayview Village
CFB Toronto
Riverdale, Toronto
Design Exchange
Toronto Dominion Centre
Exhibition Place
Parkdale Village
Clairlea
Golden Mile, Toronto
Oakridge, Toronto
York

In [15]:
print(toronto_venues.shape)
toronto_venues.head()

(1297, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


Let's check how many venues were returned for each neighborhood

In [19]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt North,3,3,3,3,3,3
"Agincourt, Toronto",4,4,4,4,4,4
"Alderwood, Toronto",9,9,9,9,9,9
Bathurst Manor,17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4
Beaumond Heights,9,9,9,9,9,9
"Bedford Park, Toronto",20,20,20,20,20,20
Berczy Park,20,20,20,20,20,20
Birch Cliff,4,4,4,4,4,4


## Analyze Each Neighborhood

In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [21]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Theme Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint
0,Agincourt North,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.0,0.00,0.00,0.000000,0.000000,0.0000,0.00,0.0
1,"Agincourt, Toronto",0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.0,0.00,0.00,0.000000,0.000000,0.0000,0.00,0.0
2,"Alderwood, Toronto",0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.0,0.00,0.00,0.000000,0.000000,0.0000,0.00,0.0
3,Bathurst Manor,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.0,0.00,0.00,0.058824,0.000000,0.0000,0.00,0.0
4,Bayview Village,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.0,0.00,0.00,0.000000,0.000000,0.0000,0.00,0.0
5,Beaumond Heights,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.0,0.00,0.00,0.000000,0.000000,0.0000,0.00,0.0
6,"Bedford Park, Toronto",0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,...,0.00,0.0,0.0,0.00,0.00,0.000000,0.000000,0.0000,0.00,0.0
7,Berczy Park,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.0,0.00,0.05,0.000000,0.000000,0.0000,0.00,0.0
8,Birch Cliff,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.0,0.00,0.00,0.000000,0.000000,0.0000,0.00,0.0
9,CFB Toronto,0.0000,0.0,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.0,0.0,0.00,0.00,0.000000,0.000000,0.0000,0.00,0.0


Let's confirm new size

In [22]:
toronto_grouped.shape

(129, 195)

let's write a function to sort the venues in descending order.

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [73]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
hood_venues_sorted = pd.DataFrame(columns=columns)
hood_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    hood_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

hood_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt North,Park,Playground,Asian Restaurant,Wings Joint,Construction & Landscaping,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Curling Ice
1,"Agincourt, Toronto",Skating Rink,Lounge,Sandwich Place,Breakfast Spot,Wings Joint,Cosmetics Shop,Department Store,Deli / Bodega,Dance Studio,Curling Ice
2,"Alderwood, Toronto",Pizza Place,Pub,Pharmacy,Gym,Sandwich Place,Pool,Coffee Shop,Skating Rink,Concert Hall,Dance Studio
3,Bathurst Manor,Coffee Shop,Sandwich Place,Pharmacy,Bridal Shop,Supermarket,Sushi Restaurant,Restaurant,Fast Food Restaurant,Diner,Pizza Place
4,Bayview Village,Chinese Restaurant,Japanese Restaurant,Bank,Café,Cosmetics Shop,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega,Dance Studio


## Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [74]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [75]:
# add clustering labels
hood_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

hood_merged = hood_df

# merge toronto_grouped with hood_df to add latitude/longitude for each neighborhood
hood_merged = hood_merged.join(hood_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

hood_merged.head()# check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Fast Food Restaurant,Food & Drink Shop,Wings Joint,Convenience Store,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Curling Ice
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Hockey Arena,Coffee Shop,Portuguese Restaurant,Intersection,Convenience Store,Dessert Shop,Department Store,Deli / Bodega,Dance Studio,Curling Ice
2,M5A,Downtown Toronto,Harbourfront (Toronto),43.654260,-79.360636,0.0,Coffee Shop,Breakfast Spot,Park,Historic Site,Dessert Shop,Chocolate Shop,Spa,Restaurant,Bakery,Farmers Market
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,0.0,Coffee Shop,Breakfast Spot,Park,Historic Site,Dessert Shop,Chocolate Shop,Spa,Restaurant,Bakery,Farmers Market
4,M6A,North York,Lawrence Heights,43.718518,-79.464763,0.0,Clothing Store,Furniture / Home Store,Accessories Store,Vietnamese Restaurant,Coffee Shop,Boutique,Event Space,Miscellaneous Shop,Wings Joint,Creperie


Let's visualize the clusters

In [76]:
type(hood_merged.index)

pandas.core.indexes.numeric.Int64Index

In [91]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hood_merged['Latitude'], hood_merged['Longitude'], hood_merged['Neighborhood'], hood_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[:-1],
            fill=True,
            fill_color=rainbow[:-1],
            fill_opacity=0.4).add_to(map_clusters)

map_clusters